# Imports

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import math
from collections import defaultdict
from tqdm import tqdm
import json
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from collections import defaultdict
import random
import pandas as pd


# Load Data

In [4]:
df = pd.read_csv('../dataset/weighted_score_above_08.csv')

df.head()

/var/folders/sr/kk6cvpy92zvfzcj5gr5l6c6h0000gn/T/ipykernel_12916/1413335316.py:1: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../dataset/weighted_score_above_08.csv')


,recommendationid,appid,game,author_steamid,author_num_games_owned,author_num_reviews,author_playtime_forever,author_playtime_last_two_weeks,author_playtime_at_review,author_last_played,...,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,hidden_in_steam_china,steam_china_location
0,147449116,10,Counter-Strike,76561199183984450,51,12,2548,0,2480,1696305457,...,1,99,2,0.889438,0,1,0,0,1,NaN
1,147374264,10,Counter-Strike,76561198099573060,226,13,2369,0,2361,1696096555,...,1,122,8,0.914834,0,1,0,0,1,NaN
2,147357703,10,Counter-Strike,76561199080026894,118,23,13501,212,12957,1697630734,...,1,599,20,0.968375,3,1,0,0,1,NaN
3,147345102,10,Counter-Strike,76561198068970227,28,1,10668,640,9906,1698261011,...,1,59,12,0.826206,0,0,0,0,1,NaN
4,147284743,10,Counter-Strike,76561199137893460,19,5,543,10,526,1697810991,...,1,128,24,0.853612,0,1,0,0,1,NaN


In [5]:
with open('combined_df.json', 'r') as f:
    sample_data = [json.loads(line) for line in f]

In [6]:
# Extract valid appids from the sample file
valid_appids = {entry['appid'] for entry in sample_data}

# Filter df based on appids in the sample
df = df[df['appid'].isin(valid_appids)]

# Continue processing
df_games = df[['appid', 'game']]
df_games = df_games.drop_duplicates(subset='appid')

appid2game = df_games.set_index('appid')['game'].to_dict()
game2appid = df_games.set_index('game')['appid'].to_dict()

del df_games

df = df[['appid', 'author_steamid', 'voted_up']]

In [7]:
appid2genres = {entry['appid']: entry['genre'] for entry in sample_data}

In [8]:
appid2game # dictionary from appid to game name
appid2genres # dictionary from appid to list of string genres
df.columns
# Index(['appid', 'author_steamid', 'voted_up'], dtype='object')

Index(['appid', 'author_steamid', 'voted_up'], dtype='object')

# Validation Methodology

In [9]:
# df_pos = df[df['voted_up'] == 1]
all_appids = set(df['appid'].unique())

df_pos = df[df['voted_up'] == 1].copy()

# 1) Build *global* mappings on the entire positive set
all_users = df_pos['author_steamid'].unique()
all_items = df_pos['appid'].unique()

user_ids     = {u: idx for idx, u in enumerate(all_users)}
item_ids     = {i: idx for idx, i in enumerate(all_items)}
user_ids_inv = {idx: u for u, idx in user_ids.items()}
item_ids_inv = {idx: i for i, idx in item_ids.items()}

# Create train/test split by user
def train_test_split_per_user(df, test_size=0.2, seed=42):
    np.random.seed(seed)
    train_rows, test_rows = [], []

    for user, user_df in tqdm(df.groupby('author_steamid'), desc="Splitting train/test by user"):
        if len(user_df) < 2:
            train_rows.append(user_df)
        else:
            user_train, user_test = train_test_split(user_df, test_size=test_size)
            train_rows.append(user_train)
            test_rows.append(user_test)

    train_df = pd.concat(train_rows)
    test_df = pd.concat(test_rows)
    return train_df, test_df

def sample_negatives(test_df, train_df, num_negatives=50):
    user_pos_train = train_df.groupby('author_steamid')['appid'].apply(set).to_dict()
    user_pos_test = test_df.groupby('author_steamid')['appid'].apply(set).to_dict()
    
    test_data = []

    for user in tqdm(user_pos_test, desc="Sampling negatives"):
        pos_items = user_pos_test[user]
        neg_items = list(all_appids - user_pos_train.get(user, set()) - pos_items)
        sampled_negatives = random.sample(neg_items, min(num_negatives, len(neg_items)))
        
        for pos_item in pos_items:
            test_data.append((user, pos_item, 1))
        for neg_item in sampled_negatives:
            test_data.append((user, neg_item, 0))
    
    return pd.DataFrame(test_data, columns=['author_steamid', 'appid', 'label'])

In [10]:
df_pos = df[df['voted_up'] == 1].copy()

# 1) Build *global* mappings on the entire positive set
all_users = df_pos['author_steamid'].unique()
all_items = df_pos['appid'].unique()

user_ids     = {u: idx for idx, u in enumerate(all_users)}
item_ids     = {i: idx for idx, i in enumerate(all_items)}
user_ids_inv = {idx: u for u, idx in user_ids.items()}
item_ids_inv = {idx: i for i, idx in item_ids.items()}

In [11]:
user_to_true_cache = {}

In [35]:
def dcg_at_k(relevance, k):
    relevance = np.array(relevance)[:k]
    if relevance.size == 0:
        return 0.0
    return np.sum(relevance / np.log2(np.arange(2, relevance.size + 2)))

def ndcg_at_k(preds, true_items, k):
    relevance = [1 if item in true_items else 0 for item in preds[:k]]
    ideal_relevance = sorted(relevance, reverse=True)
    dcg = dcg_at_k(relevance, k)
    idcg = dcg_at_k(ideal_relevance, k)
    return dcg / idcg if idcg > 0 else 0.0

def average_precision_at_k(preds, true_items, k):
    hits, score = 0, 0.0
    for i, item in enumerate(preds[:k]):
        if item in true_items:
            hits += 1
            score += hits / (i + 1)
    if not true_items:
        return 0.0
    return score / min(len(true_items), k)

def evaluate(test_data, recommender_fn, k=10, use_cache=False):
    global user_to_true_cache
    user_to_preds = defaultdict(list)

    if use_cache and 'user_to_true_cache' in globals():
        user_to_true = user_to_true_cache
    else:
        user_to_true = defaultdict(set)
        for _, row in tqdm(test_data.iterrows(), total=len(test_data), desc="Processing test data"):
            user, appid, label = row['author_steamid'], row['appid'], row['label']
            if label == 1:
                user_to_true[user].add(appid)
        if use_cache:
            user_to_true_cache = user_to_true

    n = 0
    for user in tqdm(user_to_true, desc="Generating predictions"):
        n += 1
        preds = recommender_fn(user)[:k]
        user_to_preds[user] = preds

    # print(f"Success rate: {successes / n:.2%}")
    # input()
    precisions, recalls, ndcgs, mapks = [], [], [], []

    for user in tqdm(user_to_true, desc="Calculating metrics"):
        true_items = user_to_true[user]
        pred_items = user_to_preds[user]

        num_hit = len(set(pred_items) & true_items)
        precision = num_hit / k
        recall = num_hit / len(true_items) if len(true_items) > 0 else 0
        ndcg = ndcg_at_k(pred_items, true_items, k)
        mapk = average_precision_at_k(pred_items, true_items, k)

        precisions.append(precision)
        recalls.append(recall)
        ndcgs.append(ndcg)
        mapks.append(mapk)

    results = {
        'Precision@K': np.mean(precisions),
        'Recall@K': np.mean(recalls),
        'NDCG@K': np.mean(ndcgs),
        'MAP@K': np.mean(mapks)
    }
    return results

# Model Experiments

## Popularity-Based

In [ ]:
train_df, test_df = train_test_split_per_user(df_pos)
test_data = sample_negatives(test_df, train_df)

In [ ]:
# Count how many users liked each item in the training data
item_popularity = train_df['appid'].value_counts()

# Recommend the top-N popular items for all users
top_k = 10
top_items = item_popularity.head(top_k).index.tolist()

# For each user, generate recommendations
def recommend_popular(user_id):
    return top_items


In [63]:
results = evaluate(test_data, recommend_popular, k=10)

for metric, value in results.items():
    print(f"{metric}: {value:.4f}")

Calculating metrics: 100%|██████████| 39976/39976 [00:00<00:00, 114204.73it/s]

Precision@K: 0.0043
Recall@K: 0.0384
NDCG@K: 0.0199
MAP@K: 0.0117


## Collaborative Filtering - Implicit ALS

In [14]:
from scipy.sparse import coo_matrix
import implicit

/opt/homebrew/Caskroom/miniconda/base/envs/mgt_flask/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
def leave_one_out_split(df, user_col='author_steamid', seed=42):
    np.random.seed(seed)
    test_indices = []

    # Collect one test index per user with >1 interaction
    for user, group in tqdm(df.groupby(user_col), desc="Leave-one-out (fast)"):
        if len(group) > 1:
            test_idx = np.random.choice(group.index, size=1)[0]
            test_indices.append(test_idx)

    # Convert to sets for fast lookup
    test_indices_set = set(test_indices)
    
    # Use Boolean mask for efficient slicing
    mask = df.index.isin(test_indices_set)
    test_df  = df.loc[mask].reset_index(drop=True)
    train_df = df.loc[~mask].reset_index(drop=True)
    
    return train_df, test_df

In [16]:
train_loo_df, test_loo_df = leave_one_out_split(df_pos, seed=42)

Leave-one-out (fast): 100%|██████████| 320412/320412 [00:01<00:00, 166237.85it/s]


In [ ]:
# Step 1: Build user/item mappings from the ENTIRE dataset (not just training)
all_user_ids = {uid: idx for idx, uid in enumerate(df_pos['author_steamid'].unique())}
all_item_ids = {iid: idx for idx, iid in enumerate(df_pos['appid'].unique())}
all_item_ids_inv = {v: k for k, v in all_item_ids.items()}

# Step 2: Add index columns to the entire dataset
df_pos['user_idx'] = df_pos['author_steamid'].map(all_user_ids)
df_pos['item_idx'] = df_pos['appid'].map(all_item_ids)

# Step 3: Now split AFTER indexing
train_df, test_df = train_test_split_per_user(df_pos)

# Step 4: Build matrix only from train data, but use global user/item shapes
item_user_matrix = coo_matrix(
    (np.ones(len(train_df)), (train_df['item_idx'], train_df['user_idx'])),
    shape=(len(all_item_ids), len(all_user_ids))
).tocsr()


## Content-Aware: LightFM With Genres

In [ ]:
train_df, test_df = train_test_split_per_user(df_pos)  # same as before

Splitting train/test by user: 100%|██████████| 320412/320412 [00:11<00:00, 28447.29it/s]


In [19]:
from lightfm import LightFM
from lightfm.data import Dataset

dataset = Dataset()

# Fit users, items, and item features (genres)
dataset.fit(train_df['author_steamid'], appid2genres.keys())
all_genres = set(g for genres in appid2genres.values() for g in genres)
dataset.fit_partial(items=train_df['appid'], item_features=all_genres)

# Build interactions
interactions, _ = dataset.build_interactions(
    ((row['author_steamid'], row['appid']) for _, row in train_df.iterrows())
)

# Build item features
item_features = dataset.build_item_features(
    ((appid, appid2genres.get(appid, [])) for appid in appid2genres)
)

In [20]:
# Train model
model_lfm = LightFM(loss='warp')
model_lfm.fit(interactions, item_features=item_features, epochs=10)


In [21]:
def recommend_lightfm(user_id, k=10):
    if user_id not in user_ids:
        return top_items  # fallback

    user_idx = user_ids[user_id]
    scores = model_lfm.predict(user_ids[user_id], np.arange(len(item_ids)), item_features=item_features)
    top_indices = np.argsort(-scores)[:k]
    return [item_ids_inv[i] for i in top_indices]


In [23]:
results = evaluate(test_data, recommend_lightfm, k=10)

for metric, value in results.items():
    print(f"{metric}: {value:.4f}")

Generating predictions: 100%|██████████| 39976/39976 [01:15<00:00, 531.70it/s]


Success rate: 100.00%


Calculating metrics: 100%|██████████| 39976/39976 [00:00<00:00, 110374.24it/s]

Precision@K: 0.0001
Recall@K: 0.0004
NDCG@K: 0.0003
MAP@K: 0.0002
